In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
from collections import defaultdict

ModuleNotFoundError: No module named 'numpy'

In [2]:
train_data=pd.read_csv('../gowalla/Tourist_place1.csv')

NameError: name 'pd' is not defined

In [3]:
train_data.head()

NameError: name 'train_data' is not defined

In [4]:
def read_training_data():
    size_file ="../Dataset2/Gowalla_data_size.txt"
    user_num, poi_num = open(size_file, 'r').readlines()[0].strip('\n').split()
    user_num, poi_num = int(user_num), int(poi_num)
    train_data = open('../Dataset2/Gowalla_train.txt', 'r').readlines()
    training_matrix = np.zeros((user_num, poi_num))
    count=0
    for eachline in train_data:
        uid, lid, _ = eachline.strip().split()
        uid, lid = int(uid), int(lid)
        training_matrix[uid, lid] = 1.0
        ++count
        if(count==5000):
            break
    return training_matrix

In [5]:
read_training_data()

NameError: name 'np' is not defined

In [6]:
processdata=np.zeros((train_data.user_id.unique().shape[0],train_data.place_id.unique().shape[0]))

NameError: name 'np' is not defined

In [159]:
for x,checkin in train_data.iterrows():
    print(checkin[0],checkin[1])
    processdata[checkin[0]][checkin[1]]=checkin[4]

1 3
2 5
3 4
4 2
5 1
1 4
2 2
3 3
0 0
0 4


In [35]:
print(processdata)

[[1. 0. 0. 0. 2. 0.]
 [0. 0. 0. 1. 2. 0.]
 [0. 0. 1. 0. 0. 2.]
 [0. 0. 0. 1. 2. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [160]:
def pre_compute_rec_scores(C):
        sim = C.dot(C.T)
        sim=sim/(norm(C,axis=1)*norm(C.T,axis=0))
        print(sim)
        norms = [norm(C[i]) for i in range(C.shape[0])]
        print(norms)
        for i in range(C.shape[0]):
            sim[i][i] = 0.0
            for j in range(i+1, C.shape[0]):
                sim[i][j] /= (norms[i] * norms[j])
                sim[j][i] /= (norms[i] * norms[j])

        rec_score = sim.dot(C)
       
        return sim

In [161]:
pre_compute_rec_scores(processdata)

[[1.  0.8 0.  0.8 0.  0. ]
 [0.8 1.  0.  1.  0.  0. ]
 [0.  0.  1.  0.  1.  0. ]
 [0.8 1.  0.  1.  0.  0. ]
 [0.  0.  0.2 0.  1.  0. ]
 [0.  0.  0.  0.  0.  1. ]]
[2.23606797749979, 2.23606797749979, 2.23606797749979, 2.23606797749979, 1.0, 1.0]


array([[0.        , 0.16      , 0.        , 0.16      , 0.        ,
        0.        ],
       [0.16      , 0.        , 0.        , 0.2       , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.4472136 ,
        0.        ],
       [0.16      , 0.2       , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.08944272, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In [162]:
cosine_similarity(processdata)

array([[1.       , 0.8      , 0.       , 0.8      , 0.       , 0.       ],
       [0.8      , 1.       , 0.       , 1.       , 0.       , 0.       ],
       [0.       , 0.       , 1.       , 0.       , 0.4472136, 0.       ],
       [0.8      , 1.       , 0.       , 1.       , 0.       , 0.       ],
       [0.       , 0.       , 0.4472136, 0.       , 1.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 1.       ]])

In [163]:
def cosine_similarity(train_matrix, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = train_matrix.dot(train_matrix.T) 
    elif kind == 'location':
        sim = train_matrix.T.dot(train_matrix) 
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

user_similarity = cosine_similarity(processdata, kind='user')
print(user_similarity)

[[1.        0.8       0.        0.8       0.        0.       ]
 [0.8       1.        0.        1.        0.        0.       ]
 [0.        0.        1.        0.        0.4472136 0.       ]
 [0.8       1.        0.        1.        0.        0.       ]
 [0.        0.        0.4472136 0.        1.        0.       ]
 [0.        0.        0.        0.        0.        1.       ]]


In [164]:
def predict(checkins, similarity, type='user'):
    if type == 'user':
        pred = similarity.dot(checkins)/np.array([np.abs(similarity).sum(axis=1)]).T        
    elif type == 'item':
        pred = checkins.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
        
    return pred

user_prediction = predict(processdata, user_similarity, type='user')
print(user_prediction)

[[0.38461538 0.         0.         0.61538462 2.         0.        ]
 [0.28571429 0.         0.         0.71428571 2.         0.        ]
 [0.         0.         1.         0.         0.         1.38196601]
 [0.28571429 0.         0.         0.71428571 2.         0.        ]
 [0.         0.         1.         0.         0.         0.61803399]
 [0.         1.         0.         0.         0.         0.        ]]


In [165]:
def read_ground_truth():
    ground_truth = defaultdict(set)
    dataset=pd.read_csv('../gowalla/test.csv')
    for x,checkin in dataset.iterrows():
        ground_truth[checkin[0]].add(checkin[1])
    return ground_truth

In [166]:
ground_truth=read_ground_truth()

In [167]:
def precision_k(actual, predicted):
    return 1.0 * len(set(actual) & set(predicted)) / len(predicted)


def recall_k(actual, predicted):
    return 1.0 * len(set(actual) & set(predicted)) / len(actual)

In [168]:
def normalize(scores):
    max_score = max(scores)
    if not max_score == 0:
        scores = [s / max_score for s in scores]
    return scores

In [7]:
dataset2=pd.read_csv('../gowalla/test.csv')
user_num, poi_num =dataset2.user_id.unique().shape[0] ,dataset2.place_id.unique().shape[0]
all_uids = list(range(user_num))
all_lids = list(range(poi_num))
np.random.shuffle(all_uids)
precision, recall = [], []
alpha = 0.1
beta = 0.1
for cnt, uid in enumerate(all_uids):
        if uid in ground_truth:
            U_scores = normalize([user_prediction[uid][lid]
                                  if processdata[uid, lid] == 0 else 1
                                  for lid in all_lids])
        

            U_scores = np.array(U_scores)
        
            overall_scores = (1.0 - alpha - beta) * U_scores + alpha * 0 + beta * 0

            predicted = list(reversed(overall_scores.argsort()))[:6]
            actual = ground_truth[uid]

            precision.append(precision_k(actual, predicted[:6]))
            recall.append(recall_k(actual, predicted[:3]))
            print(cnt,uid, "pre: ", np.mean(precision), "rec: ", np.mean(recall))
            print("user_id: "+str(uid))
            print("Location_id:",np.array([lid for lid in predicted]))
            count=0
            for lid in predicted:
                count+=1
                print("Rank:"+str(count),train_data.loc[train_data['place_id']==lid ,'place_name'][:1])
            print("\n")
           
            

NameError: name 'pd' is not defined